# ESSE 2220 – Lab 6 Report
## LED Matrix Feature Detection & Visualization

**Course:** ESSE 2220  
**Lab Title:** LED Matrix Feature Detection & Visualization  
**Date Performed:** 2025-10-31  
**Date Submitted:** 2025-11-04

---
## 1. Names & Group Info
- **Group Number:** 5
- **Members:** Yathharthha Kaushal, Owen

---

## 2. Setup

### 2.1 Circuit Photo

![Circuit Photo](https://raw.githubusercontent.com/YK12321/ESSE2220-labs/main/6/circuit.jpg)

*The circuit shows the Raspberry Pi connected to the 8×8 LED matrix.*

### 2.2 Python Code

#### 2.2.1 Image Processing Code (OpenCV)

In [ ]:
import cv2
import numpy as np

def loadImage(path):
    """Load an image from the specified file path."""
    image = cv2.imread(path)
    if image is None:
        raise FileNotFoundError(f"Image not found at path: {path}")
    return image

def resizeImage(image, size=(8, 8)):
    """Resize image to specified dimensions (default 8x8 for LED matrix).
    Uses INTER_AREA interpolation for better downsampling quality."""
    resized = cv2.resize(image, size, interpolation=cv2.INTER_AREA)
    return resized

def sobel(image):
    """Apply Sobel edge detection to the image.
    Computes gradients in X and Y directions, then combines them using magnitude."""
    # Use ksize=3 for small images (5 is too large for 8x8)
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)  # Horizontal edges
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)  # Vertical edges
    # Combine X and Y gradients using magnitude
    sobel_combined = cv2.magnitude(sobelx, sobely)
    sobel_combined = np.uint8(np.clip(sobel_combined, 0, 255))
    return sobel_combined

def canny(image):
    """Apply Canny edge detection to the image.
    Uses hysteresis thresholding with lower threshold=100, upper threshold=200."""
    edges = cv2.Canny(image, 100, 200)
    return edges

def convertToBinary(image, threshold=127, invert=False):
    """Convert a grayscale image to a binary image using the specified threshold.
    
    Args:
        image: Grayscale image
        threshold: Pixel values above this become 1 (white), below become 0 (black)
        invert: If True, invert the binary output (1 becomes 0, 0 becomes 1)
    """
    if invert:
        # Inverted: dark pixels in original → LED on (1)
        _, binary_image = cv2.threshold(image, threshold, 1, cv2.THRESH_BINARY_INV)
    else:
        # Normal: bright pixels in original → LED on (1)
        _, binary_image = cv2.threshold(image, threshold, 1, cv2.THRESH_BINARY)
    return binary_image

def convertToHex(binary_image, reverse_bits=False):
    """Convert a binary image to hexadecimal representation for LED matrix.
    Each row of 8 pixels becomes one byte (hex value).
    
    Args:
        binary_image: 8x8 binary image where 1=LED on, 0=LED off
        reverse_bits: If True, reverse the bit order in each row (MSB<->LSB)
    """
    hex_data = []
    print("\n=== Binary to Hex Conversion ===")
    for row_idx, row in enumerate(binary_image):
        # Ensure we only take 8 bits (8 pixels) per row
        row_flat = row.flatten() if len(row.shape) > 1 else row
        # Take only first 8 bits if row is longer
        row_8bit = row_flat[:8] if len(row_flat) > 8 else row_flat
        # Pad with zeros if row is shorter than 8 bits
        if len(row_8bit) < 8:
            row_8bit = np.pad(row_8bit, (0, 8 - len(row_8bit)), 'constant')
        
        # Reverse bits if needed (for LED matrices with different scan directions)
        if reverse_bits:
            row_8bit = row_8bit[::-1]
        
        # Convert binary array to bit string
        bits = ''.join(str(int(bit)) for bit in row_8bit)
        # Convert bit string to integer (base 2)
        hex_value = int(bits, 2)
        hex_data.append(hex_value)
        print(f"Row {row_idx}: {bits} = {hex_value:3d} = 0x{hex_value:02x}")
    print("================================\n")
    return hex_data

if __name__ == '__main__':
    print('Program is starting...')
    try:
        # Load the input image
        image = loadImage("lab6_8x8_gray.png")
        gray_image = image  # Image provided is already grayscale
        
        # FIRST: Resize to 8x8 (LED matrix dimensions)
        resized_8x8 = resizeImage(gray_image, (8, 8))
        
        # THEN: Apply edge detection algorithms on the 8x8 image
        sobel_8x8 = sobel(resized_8x8)
        canny_8x8 = canny(resized_8x8)
        
        # Create upscaled versions for better viewing (scale 8x8 to 240x240)
        scale_factor = 30
        original_large = cv2.resize(resized_8x8, (8 * scale_factor, 8 * scale_factor), 
                                   interpolation=cv2.INTER_NEAREST)
        sobel_large = cv2.resize(sobel_8x8, (8 * scale_factor, 8 * scale_factor), 
                                interpolation=cv2.INTER_NEAREST)
        canny_large = cv2.resize(canny_8x8, (8 * scale_factor, 8 * scale_factor), 
                                interpolation=cv2.INTER_NEAREST)
        
        # Print pixel value ranges for debugging
        print("\n=== Pixel Value Ranges (for debugging) ===")
        print(f"Original 8x8:   min={resized_8x8.min()}, max={resized_8x8.max()}")
        print(f"Sobel 8x8:      min={sobel_8x8.min()}, max={sobel_8x8.max()}")
        print(f"Canny 8x8:      min={canny_8x8.min()}, max={canny_8x8.max()}")
        
        # Convert all methods to binary for comparison
        # Set invert=True if you want dark pixels in image = LED on
        invert_binary = True  # Change to False if LEDs should match white pixels
        
        binary_original = convertToBinary(resized_8x8, threshold=127, invert=invert_binary)
        binary_sobel = convertToBinary(sobel_8x8, threshold=127, invert=False)
        binary_canny = convertToBinary(canny_8x8, threshold=127, invert=False)
        
        print(f"Binary original (inverted={invert_binary}): min={binary_original.min()}, max={binary_original.max()}")
        print(f"Binary sobel:    min={binary_sobel.min()}, max={binary_sobel.max()}")
        print(f"Binary canny:    min={binary_canny.min()}, max={binary_canny.max()}")
        
        # Print actual 8x8 binary pattern for original
        print("\n8x8 Binary Original Pattern (1=LED on, 0=LED off):")
        for row in binary_original:
            row_flat = row.flatten() if len(row.shape) > 1 else row
            print(''.join(str(int(pixel)) for pixel in row_flat))
        print("==========================================\n")
        
        # Create scaled versions for display
        binary_original_large = cv2.resize(binary_original * 255, 
                                          (8 * scale_factor, 8 * scale_factor), 
                                          interpolation=cv2.INTER_NEAREST)
        binary_sobel_large = cv2.resize(binary_sobel * 255, 
                                       (8 * scale_factor, 8 * scale_factor), 
                                       interpolation=cv2.INTER_NEAREST)
        binary_canny_large = cv2.resize(binary_canny * 255, 
                                       (8 * scale_factor, 8 * scale_factor), 
                                       interpolation=cv2.INTER_NEAREST)
        
        # Save all images to files for documentation
        cv2.imwrite("output_original_8x8.png", original_large)
        cv2.imwrite("output_sobel_8x8.png", sobel_large)
        cv2.imwrite("output_canny_8x8.png", canny_large)
        cv2.imwrite("output_binary_original.png", binary_original_large)
        cv2.imwrite("output_binary_sobel.png", binary_sobel_large)
        cv2.imwrite("output_binary_canny.png", binary_canny_large)
        
        print("Images saved (all scaled up 30x for viewing):")
        print("  - output_original_8x8.png (Original resized to 8x8)")
        print("  - output_sobel_8x8.png (Sobel edge detection on 8x8)")
        print("  - output_canny_8x8.png (Canny edge detection on 8x8)")
        print("  - output_binary_original.png (Binary of original)")
        print("  - output_binary_sobel.png (Binary of Sobel - for LED)")
        print("  - output_binary_canny.png (Binary of Canny - for LED)")
        
        # Display images in windows
        cv2.imshow("1. Original 8x8", original_large)
        cv2.imshow("2. Sobel 8x8", sobel_large)
        cv2.imshow("3. Canny 8x8", canny_large)
        cv2.imshow("4. Binary Original", binary_original_large)
        cv2.imshow("5. Binary Sobel", binary_sobel_large)
        cv2.imshow("6. Binary Canny", binary_canny_large)
        
        # Convert saved images to hex data for LED matrix
        print("\n=== Converting SAVED images back to hex data ===")
        
        # Read back the saved binary images
        saved_binary_original = cv2.imread("output_binary_original.png", cv2.IMREAD_GRAYSCALE)
        saved_binary_sobel = cv2.imread("output_binary_sobel.png", cv2.IMREAD_GRAYSCALE)
        saved_binary_canny = cv2.imread("output_binary_canny.png", cv2.IMREAD_GRAYSCALE)
        
        # Downscale back to 8x8
        saved_binary_original_8x8 = cv2.resize(saved_binary_original, (8, 8), 
                                              interpolation=cv2.INTER_NEAREST)
        saved_binary_sobel_8x8 = cv2.resize(saved_binary_sobel, (8, 8), 
                                           interpolation=cv2.INTER_NEAREST)
        saved_binary_canny_8x8 = cv2.resize(saved_binary_canny, (8, 8), 
                                           interpolation=cv2.INTER_NEAREST)
        
        # Convert back to binary (0 and 1)
        _, saved_binary_original_8x8 = cv2.threshold(saved_binary_original_8x8, 127, 1, 
                                                     cv2.THRESH_BINARY)
        _, saved_binary_sobel_8x8 = cv2.threshold(saved_binary_sobel_8x8, 127, 1, 
                                                  cv2.THRESH_BINARY)
        _, saved_binary_canny_8x8 = cv2.threshold(saved_binary_canny_8x8, 127, 1, 
                                                  cv2.THRESH_BINARY)
        
        # Convert to hex data for LED matrix
        reverse_bits = False  # Change to True if display is horizontally flipped
        
        print("\n--- Converting BINARY to hex ---")
        hex_binary = convertToHex(saved_binary_original_8x8, reverse_bits)
        print("\n--- Converting SOBEL to hex ---")
        hex_sobel = convertToHex(saved_binary_sobel_8x8, reverse_bits)
        print("\n--- Converting CANNY to hex ---")
        hex_canny = convertToHex(saved_binary_canny_8x8, reverse_bits)
        
        # Display in format compatible with ledMatrix.py
        print("\nHexadecimal representation for LED Matrix:")
        print(f"binarypic = {hex_binary}")
        print(f"sobelpic = {hex_sobel}")
        print(f"cannypic = {hex_canny}")

        # Save to file for easy copying to LED matrix code
        with open("imageHexData.dat", "w") as f:
            f.write(f"binarypic = {hex_binary}\n")
            f.write(f"sobelpic = {hex_sobel}\n")
            f.write(f"cannypic = {hex_canny}\n")
        
        print("\nData saved to imageHexData.dat")
        print(f"Number of rows: {len(hex_sobel)}")
        
        # Wait for key press to close windows
        print("\nPress any key in the image window to close...")
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    except FileNotFoundError as e:
        print(e)
    except Exception as e:
        print(f'Error: {e}')

#### 2.2.2 LED Matrix Display Code

In [ ]:
import RPi.GPIO as GPIO
import time

LSBFIRST = 1
MSBFIRST = 2

# Define the pins connected to 74HC595 shift registers
dataPin   = 11      # DS Pin of 74HC595(Pin14) - Serial data input
latchPin  = 13      # ST_CP Pin of 74HC595(Pin12) - Storage register clock
clockPin = 15       # SH_CP Pin of 74HC595(Pin11) - Shift register clock

def reverseBits(byte):
    """Reverse the bits in a byte (mirror horizontally).
    This corrects horizontal flipping in the LED display."""
    result = 0
    for i in range(8):
        if byte & (1 << i):
            result |= (1 << (7 - i))
    return result

def mirrorHorizontal(pattern):
    """Mirror a pattern horizontally by reversing bits in each row."""
    return [reverseBits(byte) for byte in pattern]

def rotateLeft90(pattern):
    """Rotate pattern 90 degrees counter-clockwise (left).
    This corrects the orientation mismatch between OpenCV and LED display."""
    # Convert bytes to 8x8 bit array
    bits = []
    for byte in pattern:
        row = [(byte >> (7-i)) & 1 for i in range(8)]
        bits.append(row)
    
    # Rotate: new[row][col] = old[col][7-row]
    rotated = [[bits[col][7-row] for col in range(8)] for row in range(8)]
    
    # Convert back to bytes
    result = []
    for row in rotated:
        byte = 0
        for i, bit in enumerate(row):
            byte |= (bit << (7-i))
        result.append(byte)
    return result

# Image data from imageHexData.dat (raw values generated by processImage.py)
binarypic_raw = [241, 130, 132, 136, 240, 240, 240, 240]
sobelpic_raw = [96, 255, 127, 255, 255, 255, 255, 126]
cannypic_raw = [105, 81, 139, 80, 100, 73, 136, 41]

# Fix orientation: mirror horizontally then rotate left 90 degrees
# This ensures the LED display matches the OpenCV image orientation
binarypic = rotateLeft90(mirrorHorizontal(binarypic_raw))
sobelpic = rotateLeft90(mirrorHorizontal(sobelpic_raw))
cannypic = rotateLeft90(mirrorHorizontal(cannypic_raw))

# Letter patterns for labeling (B=Binary, S=Sobel, C=Canny)
letter_B = [0x00, 0x00, 0x7F, 0x49, 0x49, 0x36, 0x00, 0x00]  # "B"
letter_S_raw = [0x00, 0x00, 0x46, 0x49, 0x49, 0x31, 0x00, 0x00]  # "S" (raw)
letter_S = mirrorHorizontal(letter_S_raw)  # Fix horizontal mirror for "S"
letter_C = [0x00, 0x00, 0x3E, 0x41, 0x41, 0x22, 0x00, 0x00]  # "C"
blank = [0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00]     # Blank screen

# Dictionary mapping labels to their display data
data = {
    'B': letter_B,
    'binarypic': binarypic,
    'S': letter_S,
    'sobelpic': sobelpic,
    'C': letter_C,
    'cannypic': cannypic,
    'blank': blank
}

def setup():
    """Initialize GPIO pins for controlling the 74HC595 shift registers."""
    GPIO.setmode(GPIO.BOARD)    # Use physical pin numbering
    GPIO.setup(dataPin, GPIO.OUT)
    GPIO.setup(latchPin, GPIO.OUT)
    GPIO.setup(clockPin, GPIO.OUT)

def shiftOut(dPin, cPin, order, val):
    """Shift out a byte of data to the 74HC595 shift register.
    
    Args:
        dPin: Data pin (serial data input)
        cPin: Clock pin (shift register clock)
        order: LSBFIRST or MSBFIRST (bit order)
        val: 8-bit value to shift out
    """
    for i in range(0, 8):
        GPIO.output(cPin, GPIO.LOW)
        if(order == LSBFIRST):
            # Shift out LSB first
            GPIO.output(dPin, (0x01 & (val >> i) == 0x01) and GPIO.HIGH or GPIO.LOW)
        elif(order == MSBFIRST):
            # Shift out MSB first
            GPIO.output(dPin, (0x80 & (val << i) == 0x80) and GPIO.HIGH or GPIO.LOW)
        GPIO.output(cPin, GPIO.HIGH)

def displayPattern(pattern, duration):
    """Display a pattern on the LED matrix for the specified duration.
    
    Uses multiplexing to rapidly scan through rows, creating the illusion
    of all rows being lit simultaneously.
    
    Args:
        pattern: List of 8 bytes, each representing one row of the 8x8 matrix
        duration: Time in seconds to display the pattern
    """
    start_time = time.time()
    while time.time() - start_time < duration:
        x = 0x80  # Start with first row (binary 10000000)
        for i in range(0, 8):
            # Latch low to prepare for data transfer
            GPIO.output(latchPin, GPIO.LOW)
            # Shift out column data (which LEDs in this row should be on)
            shiftOut(dataPin, clockPin, MSBFIRST, pattern[i])
            # Shift out row selector (which row to activate)
            shiftOut(dataPin, clockPin, MSBFIRST, ~x)  # Inverted for common cathode
            # Latch high to display the data
            GPIO.output(latchPin, GPIO.HIGH)
            # Brief delay for multiplexing (persistence of vision)
            time.sleep(0.001)
            x >>= 1  # Move to next row

def loop():
    """Main display loop: cycles through all three edge detection methods.
    
    Sequence:
    - B (2s) → Binary Original (10s) → Blank (0.5s)
    - S (2s) → Sobel Edges (10s) → Blank (0.5s)
    - C (2s) → Canny Edges (10s) → Blank (0.5s)
    """
    while True:
        # Display Binary Original
        displayPattern(data['B'], 2)
        displayPattern(data['binarypic'], 10)
        displayPattern(data['blank'], 0.5)
        
        # Display Sobel Edge Detection
        displayPattern(data['S'], 2)
        displayPattern(data['sobelpic'], 10)
        displayPattern(data['blank'], 0.5)
        
        # Display Canny Edge Detection
        displayPattern(data['C'], 2)
        displayPattern(data['cannypic'], 10)
        displayPattern(data['blank'], 0.5)

def destroy():
    """Clean up GPIO resources before exiting."""
    GPIO.cleanup()

if __name__ == '__main__':
    print('Program is starting...')
    setup()
    try:
        loop()
    except KeyboardInterrupt:
        destroy()

---
## 3. Observations

### 3.1 Edge Detector Output Images

#### Sobel Edge Detection
![Sobel Output](https://raw.githubusercontent.com/YK12321/ESSE2220-labs/main/6/output_sobel_8x8.png)

*The Sobel edge detector computes horizontal and vertical gradients, then combines them using magnitude. It effectively highlights areas of rapid intensity change.*

#### Canny Edge Detection
![Canny Output](https://raw.githubusercontent.com/YK12321/ESSE2220-labs/main/6/output_canny_8x8.png)

*The Canny edge detector uses a multi-stage algorithm including noise reduction, gradient calculation, non-maximum suppression, and hysteresis thresholding to produce thin, connected edges.*

### 3.2 Hexadecimal Output

The processed images were converted to hexadecimal representation for the LED matrix display:

```python
# Binary Original Image
binarypic = [241, 130, 132, 136, 240, 240, 240, 240]
# Hexadecimal: [0xF1, 0x82, 0x84, 0x88, 0xF0, 0xF0, 0xF0, 0xF0]

# Sobel Edge Detection
sobelpic = [96, 255, 127, 255, 255, 255, 255, 126]
# Hexadecimal: [0x60, 0xFF, 0x7F, 0xFF, 0xFF, 0xFF, 0xFF, 0x7E]

# Canny Edge Detection
cannypic = [105, 81, 139, 80, 100, 73, 136, 41]
# Hexadecimal: [0x69, 0x51, 0x8B, 0x50, 0x64, 0x49, 0x88, 0x29]
```

Each array contains 8 bytes, where each byte represents one row of the 8×8 LED matrix. A bit value of 1 indicates the LED should be illuminated, and 0 indicates it should be off.

### 3.3 LED Matrix Display Video

**Video Demonstration:** [Watch Full LED Matrix Display Video](https://raw.githubusercontent.com/YK12321/ESSE2220-labs/main/6/led_matrix_demo.mp4)

The video demonstrates the complete display cycle of the LED matrix showing all three edge detection results:

**Display Sequence (repeating):**
1. Letter "B" (2 seconds) → Binary original image (10 seconds)
2. Blank screen transition (0.5 seconds)
3. Letter "S" (2 seconds) → Sobel edge detection (10 seconds)
4. Blank screen transition (0.5 seconds)
5. Letter "C" (2 seconds) → Canny edge detection (10 seconds)
6. Blank screen transition (0.5 seconds)

**Key Observations from Video:**
- The LED matrix successfully displays all three patterns with correct orientation
- The multiplexing technique creates smooth, flicker-free display through persistence of vision
- Labels (B, S, C) clearly identify which edge detection method is being shown
- The Sobel pattern appears denser with more illuminated LEDs, showing its characteristic thicker edges
- The Canny pattern appears sparser, displaying only the most significant edge pixels
- Transitions between patterns are clean and synchronized with the programmed timing

---
## 4. Analysis

### 4.1 Main Differences Between Edge Detectors

**Sobel Edge Detector:**
- Uses convolution with Sobel kernels to compute gradients in X and Y directions
- Produces grayscale output where intensity represents edge strength
- Relatively simple and fast, but can produce thicker edges
- Sensitive to noise and can detect false edges in noisy images

**Laplacian Edge Detector:**
- Uses second-order derivatives to detect edges
- Detects edges in all directions simultaneously (isotropic)
- Very sensitive to noise due to the second derivative
- Can detect edge locations more precisely but requires pre-filtering (e.g., Gaussian smoothing)

**Canny Edge Detector:**
- Multi-stage algorithm: Gaussian smoothing → gradient calculation → non-maximum suppression → hysteresis thresholding
- Produces binary output (edge or no edge)
- Creates thin, well-connected edges
- More computationally expensive but produces the best quality edges
- Uses two thresholds (upper and lower) to link weak edges to strong edges

### 4.2 Clearest Edge Detector for 8×8 Images

For our 8×8 image, **Sobel produced the most meaningful output**. The Sobel detector captured more edge information and provided better visibility on the LED matrix compared to Canny.

**Why Sobel performed better:**
- Sobel produces gradient magnitude values, which when thresholded, preserve more edge pixels
- The 8×8 resolution is extremely limited, and Sobel's tendency to produce thicker edges actually helps fill the small display
- Canny's edge thinning algorithm (non-maximum suppression) removes too much information, leaving sparse patterns that are difficult to interpret at such low resolution
- On the LED display, the Sobel pattern showed more recognizable features from the original image

**However**, Canny would be superior for higher resolution images where precise edge localization and thin edges are beneficial.

### 4.3 Effect of 8×8 Image Size on Edge Detection

The extremely small 8×8 resolution significantly affects edge detection accuracy and visibility:

**Challenges:**
1. **Loss of fine detail**: When downsampling to 8×8, most fine edges and textures are lost
2. **Aliasing**: Diagonal edges become jagged and difficult to represent accurately
3. **Reduced gradient information**: With only 64 pixels, gradient calculations have very limited spatial context
4. **Quantization errors**: Edge detectors work best with smooth gradients, but 8×8 images have abrupt transitions

**Impact on visibility:**
- Sharp corners and boundaries are pixelated and may not appear as distinct edges
- Thin features smaller than one pixel vanish entirely
- Edge detectors struggle to distinguish actual edges from quantization artifacts
- The binary thresholding step becomes critical—choosing the wrong threshold can eliminate all edges or fill the entire display

**Mitigation strategy:**
- We resized the image to 8×8 first, then applied edge detection, rather than detecting edges and then resizing
- This preserves the edge patterns that are actually visible at the target resolution
- Pre-processing (contrast enhancement, sharpening) before downsampling can help emphasize important features

### 4.4 LED Display Orientation Matching

**Initial observation:** The LED display orientation did NOT match the OpenCV image initially.

**Why the mismatch occurred:**
1. **Coordinate system difference**: OpenCV uses (row, column) indexing with origin at top-left, while the LED matrix hardware may scan in a different order
2. **Bit order**: The 74HC595 shift register shifts data in a specific direction (MSB or LSB first)
3. **Physical wiring**: The way the LED matrix is wired (row vs. column drivers) affects how data maps to physical LEDs
4. **Horizontal mirroring**: The display was horizontally flipped compared to the OpenCV image

**Our solution:**
We implemented two correction functions in `ledMatrix.py`:
```python
def reverseBits(byte):
    # Mirrors each row horizontally by reversing bit order
    
def rotateLeft90(pattern):
    # Rotates the entire pattern 90° counter-clockwise
```

By applying `rotateLeft90(mirrorHorizontal(pattern_raw))`, we corrected the orientation so the LED display matches the OpenCV output.

### 4.5 Effect of Canny Threshold Adjustments

The Canny edge detector uses two thresholds in its hysteresis step:
- **Lower threshold (currently 100)**: Weak edges below this are discarded
- **Upper threshold (currently 200)**: Strong edges above this are kept
- **Hysteresis**: Weak edges between the thresholds are kept only if connected to strong edges

**Effects of increasing thresholds:**
- Fewer edges detected (more selective)
- Only the strongest, most prominent edges remain
- Reduces noise and false edge detection
- May lose important features if increased too much
- Result: Sparser edge map with better precision but lower recall

**Effects of decreasing thresholds:**
- More edges detected (less selective)
- Weaker gradients are classified as edges
- Captures more detail but increases false positives
- More sensitive to noise and texture
- Result: Denser edge map with higher recall but lower precision

**For our 8×8 image:**
- Our thresholds (100, 200) work well for the small resolution
- Lower thresholds would make the Canny output too dense and noisy
- Higher thresholds would make it too sparse and lose important features
- The threshold choice becomes critical at low resolution since we have so few pixels to work with

### 4.6 Real-World Applications in Embedded/Robotic Systems

Feature extraction and LED visualization techniques like those demonstrated in this lab have numerous practical applications:

**1. Robot Vision Systems:**
- **Obstacle detection**: Edge detection helps robots identify boundaries and obstacles in real-time
- **Line following**: Simple edge detection on low-resolution cameras can guide robots along paths
- **Object recognition**: Feature extraction is the first step in identifying objects for manipulation

**2. Embedded Vision Applications:**
- **Gesture recognition**: Low-resolution edge detection can identify hand gestures for human-machine interfaces
- **Motion detection**: Comparing edge maps between frames detects movement with low computational overhead
- **Quality control**: Industrial systems use edge detection to identify defects in manufactured parts

**3. Resource-Constrained Devices:**
- **Low-power operation**: Processing 8×8 images requires minimal memory and computation, enabling battery-powered devices
- **Real-time processing**: Small images can be processed at high frame rates even on simple microcontrollers
- **IoT sensors**: Edge detection preprocesses images before transmission, reducing bandwidth requirements

**4. Medical Devices:**
- **Portable diagnostics**: Low-resolution feature extraction can screen for abnormalities before detailed analysis
- **Assistive technology**: Visual feedback on LED displays for visually impaired users

**5. Automotive Systems:**
- **Parking sensors**: Simple feature detection identifies nearby objects
- **Driver assistance**: Lane detection and object recognition for safety systems

**Our lab demonstration:** The LED matrix provides immediate visual feedback, useful for debugging vision algorithms in real-time without needing a full display—a common scenario in embedded development where you need to verify that your vision system is working correctly with minimal overhead.

---
## 5. Conclusion

In this lab, we successfully implemented feature extraction using OpenCV edge detection algorithms (Sobel and Canny) and visualized the results on an 8×8 LED matrix controlled by a Raspberry Pi. We learned:

1. **Edge detection algorithms** have different characteristics—Sobel produces thicker edges better suited for low resolution, while Canny produces precise but sparse edges
2. **Resolution constraints** significantly impact the quality and visibility of detected features
3. **Orientation correction** is necessary when interfacing between software (OpenCV) and hardware (LED matrix)
4. **Hexadecimal representation** efficiently encodes binary image data for embedded display systems
5. **Real-world applications** of feature extraction in embedded systems are widespread, from robotics to IoT

The lab demonstrated the complete pipeline from image processing to physical visualization, bridging the gap between computer vision algorithms and embedded hardware control.
